In [1]:
import pandas as pd
import numpy as np
import re
from itertools import product, combinations
from rdkit import Chem
from rdkit.Chem import Descriptors
import networkx as nx
from pyvis.network import Network
from rdkit.Chem.rdChemReactions import PreprocessReaction
from rdkit.Chem import MolFromSmiles, AllChem
from rdkit.Chem import rdChemReactions
import os

In [2]:
reaction_dir = "Storage/rxn/"
file_names = os.listdir(reaction_dir)
rxn_files = [reaction_dir + x for x in file_names]
rhea_IDs = [int(x.split(".")[0]) for x in file_names]

In [3]:
rhea2ec = pd.read_csv("Storage/rhea2ec.tsv", delimiter="\t",engine="c")

In [4]:
rheadirections = pd.read_csv("Storage/rhea-directions.tsv", delimiter="\t",engine="c")

In [5]:
rhea2chebis = pd.read_pickle("Storage/rhea_to_ChEBIs.p")

In [6]:
reactions = []
for file, ID in zip(rxn_files,rhea_IDs):
    try:
        reaction = AllChem.ReactionFromRxnFile(file)
        reactants = list(reaction.GetReactants())
        products = list(reaction.GetProducts())
        reactions.append( [reaction, reactants, products, len(reactants), len(products), ID] )
    except:
        pass # sorry

RDKit ERROR: [03:37:50] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Post-condition Violation
RDKit ERROR: Element 'hv' not found
RDKit ERROR: Violation occurred on line 91 in file /home/conda/feedstock_root/build_artifacts/rdkit_1616920171002/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR: Failed Expression: anum > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [03:37:50] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Post-condition Violation
RDKit ERROR: Element 'hv' not found
RDKit ERROR: Violation occurred on line 91 in file /home/conda/feedstock_root/build_artifacts/rdkit_1616920171002/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR: Failed Expression: anum > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [03:37:51] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Post-condition Violation
RDKit ERROR: Element 'hv' not found
RDKit ERROR: Violation occurred on line 91 in file /home/conda/feedstock_root/build_artifacts/rdkit_1616920171002/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR:

In [7]:
reactions = pd.DataFrame(reactions)
pd.DataFrame(reactions).columns = ["rdkit_rxn", "rdkit_reactants","rdkit_products","num_reactants","num_products", "rhea_ID"]

In [8]:
ChEBI_reactants = []
ChEBI_products = []
for x in range(len(reactions)):
    _rhea_ID = reactions["rhea_ID"].iloc[x]
    chebis_in_reaction = rhea2chebis[rhea2chebis["rhea_ids"] == _rhea_ID]["rhea_ChEBIs"].iloc[0]
    _ChEBI_reactants = chebis_in_reaction[reactions["num_reactants"][x]:]
    _ChEBI_products = chebis_in_reaction[:reactions["num_reactants"][x]]
    ChEBI_reactants.append(_ChEBI_reactants)
    ChEBI_products.append(_ChEBI_products)

In [9]:
reactions["ChEBI_reactants"] = ChEBI_reactants
reactions["ChEBI_products"] = ChEBI_products

In [10]:
rhea_master_rxns = []
rhea_bi_rxns = []
for x in range(len(reactions)):
    try:
        _ID = reactions["rhea_ID"].iloc[x]
        _master_rxn = rheadirections[(rheadirections["RHEA_ID_LR"] == _ID) | (rheadirections["RHEA_ID_RL"] == _ID)]["RHEA_ID_MASTER"].iloc[0]
        _bi_rxn = _master_rxn = rheadirections[(rheadirections["RHEA_ID_LR"] == _ID) | (rheadirections["RHEA_ID_RL"] == _ID)]["RHEA_ID_BI"].iloc[0]
        rhea_master_rxns.append(_master_rxn)
        rhea_bi_rxns.append(_bi_rxn)
    except:
        rhea_master_rxns.append(np.nan)
        rhea_bi_rxns.append(np.nan)

In [11]:
reactions["RHEA_ID_MASTER"] = rhea_master_rxns
reactions["RHEA_ID_BI"] = rhea_bi_rxns
reactions = reactions[~reactions["RHEA_ID_MASTER"].isna()]
reactions = reactions[~reactions["RHEA_ID_BI"].isna()]


In [12]:
reactions = reactions.drop_duplicates(subset="RHEA_ID_MASTER",keep='first')

In [13]:
reactions = reactions.loc[(reactions["ChEBI_reactants"].astype(str) + reactions["ChEBI_products"].astype(str)).drop_duplicates(keep="first").index]
reactions = reactions[~(reactions["ChEBI_reactants"].astype(str) == reactions["ChEBI_products"].astype(str))]

In [14]:
reactions.reset_index(inplace=True, drop=True)

In [24]:
adjacency_list = []
rhea_id = []
for x in range(len(reactions)):
    mini_adjacency = list(product(reactions["ChEBI_reactants"][x],reactions["ChEBI_products"][x]))
    adjacency_list.extend(mini_adjacency)
    for _ in range(len(mini_adjacency)):
        rhea_id.append(reactions["rhea_ID"][x])

In [25]:
adjacency_list = pd.DataFrame(adjacency_list)
adjacency_list.columns = ["ChEBI_reactant", "ChEBI_product"]
adjacency_list["rhea_ID"] = rhea_id

In [93]:
molecular_blacklist = [15422, 15751, 16960, 48584, 456216, 61506, 16761, 456215, 15378,15377,30616, 15379,57783,58349,57945]

In [94]:
adjacency_list = adjacency_list[~adjacency_list["ChEBI_reactant"].isin(molecular_blacklist)]
adjacency_list = adjacency_list[~adjacency_list["ChEBI_product"].isin(molecular_blacklist)]

In [95]:
adjacency_list = adjacency_list.drop_duplicates(subset=['ChEBI_reactant', 'ChEBI_product'],keep="first") # fixing stoichiometry issues

In [96]:
reactions

,rdkit_rxn,rdkit_reactants,rdkit_products,num_reactants,num_products,rhea_ID,ChEBI_reactants,ChEBI_products,RHEA_ID_MASTER,RHEA_ID_BI
0,<rdkit.Chem.rdChemReactions.ChemicalReaction o...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f328ca...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f1f9e2...,6,5,49741,"[16526, 131915, 15377, 15377, 58349]","[15378, 15378, 57783, 15379, 15379, 90907]",49743.0,49743.0
1,<rdkit.Chem.rdChemReactions.ChemicalReaction o...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f1f9e4...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f1f9e5...,4,3,53150,"[2700, 15379, 57618]","[15378, 15377, 136991, 58210]",53151.0,53151.0
2,<rdkit.Chem.rdChemReactions.ChemicalReaction o...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f1f9e7...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f1f9e9...,4,3,32905,"[28940, 15379, 57618]","[17933, 15378, 15377, 58210]",32906.0,32906.0
3,<rdkit.Chem.rdChemReactions.ChemicalReaction o...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f1f9ea...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f1fa02...,16,19,42837,"[78251, 16526, 16526, 16526, 16526, 16526, 572...","[57288, 15378, 15378, 15378, 15378, 15378, 153...",42839.0,42839.0
4,<rdkit.Chem.rdChemReactions.ChemicalReaction o...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f1fa0b...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f1fa0c...,2,4,13890,"[456215, 58210, 15378, 15378]","[57692, 15377]",13892.0,13892.0
...,...,...,...,...,...,...,...,...,...,...
12877,<rdkit.Chem.rdChemReactions.ChemicalReaction o...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f0597e...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f059a0...,4,6,33672,"[29033, 29033, 15378, 15378, 52374, 15379]","[29034, 29034, 60384, 15377]",33674.0,33674.0
12878,<rdkit.Chem.rdChemReactions.ChemicalReaction o...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f059f3...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f059f4...,3,4,49081,"[90356, 58210, 15378, 15377]","[23683, 57618, 15379]",49083.0,49083.0
12879,<rdkit.Chem.rdChemReactions.ChemicalReaction o...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f05bb6...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f05bb8...,3,3,41802,"[78446, 15378, 78449]","[78450, 16526, 64479]",41803.0,41803.0
12880,<rdkit.Chem.rdChemReactions.ChemicalReaction o...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f05469...,[<rdkit.Chem.rdchem.Mol object at 0x7fd0f0546b...,4,6,33521,"[29033, 29033, 15378, 15378, 15379, 57817]","[64124, 29034, 29034, 15377]",33522.0,33522.0


In [116]:
adjacency_list[adjacency_list["ChEBI_product"]==246256]

,ChEBI_reactant,ChEBI_product,rhea_ID
104215,101,246256,30017
104218,100101,246256,30017
104221,104100,246256,30017
104224,105102,246256,30017
104227,102103,246256,30017
...,...,...,...
105046,226231,246256,30017
105052,230233,246256,30017
105055,229234,246256,30017
105061,225,246256,30017


In [98]:
import networkx as nx

In [108]:
G = nx.from_pandas_edgelist(adjacency_list, source="ChEBI_reactant", target="ChEBI_product",edge_attr="rhea_ID")
remove = [node for node,degree in dict(G.degree()).items() if degree > 300]

In [112]:
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
G.remove_nodes_from(remove)

In [113]:
nx.write_graphml(G,"Storage/g.graphml")